## 1. 라이브러리 임포트

In [ ]:
# pandas read_excel 함수에서 엑셀을 읽기 위한 패키지 설치
!pip install openpyxl

import numpy as np
import pandas as pd

## 2. 파일 읽어오기

In [ ]:
# 데이터 읽어오기
final_data = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')

In [ ]:
final_data.head()

In [ ]:
# 총 251건
final_data.info()

## 3. 특수문자 제거

In [ ]:
# data 컬럼의 내용중에 영문, 특수문자 있는지 확인 : 영문과 특수문자 존재 확인
final_data[final_data['data'].str.contains('[^가-힣 ]')].values[:10]

In [ ]:
# data 컬럼의 내용에서 숫자, 영문자, 공백등의 글자는 삭제처리
final_data['data'] = final_data['data'].str.replace('[^가-힣 ]','')

In [ ]:
# data 컬럼의 내용에서 영문, 특수문자 없음 확인
final_data['data'][final_data['data'].str.contains('[^가-힣 ]')].sum()

## 4. 전처리

In [ ]:
# final_data 어떤 컬럼과 내용으로 되어 있는지 파악
final_data.tail()

In [ ]:
# data 컬럼의 내용을 양끝의 빈공간 삭제
final_data['data'] = final_data['data'].str.strip()

In [ ]:
final_data.tail()

In [ ]:
# 필요없는 'Unnamed: 0' 컬럼 삭제
final_data = final_data.drop(['Unnamed: 0'], axis=1)

In [ ]:
final_data.tail()

In [ ]:
# Null 있는지 확인
final_data.isnull().sum()

In [ ]:
# 중복 데이터 있는지 확인 : 2건 중복 존재 확인
final_data['data'].duplicated().sum()

In [ ]:
# 중복 데이터 제거
final_data.drop_duplicates(subset=['data'], inplace=True)

In [ ]:
# 기존 251 --> 이후 249 : 2건 중복 삭제 확인
final_data.info()

## 5. Label 분포 확인

In [ ]:
# 한글 그래프 그리기
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
plt.rc('font', family='NanumGothicCoding')

In [ ]:
# plot Bar차트 그리기
final_data['label'].value_counts().plot(kind='bar')

## 6. label 숫자로 인코딩

In [ ]:
label2class = {'긍정':0, '부정':1}
class2label = {0:'긍정', 1:'부정'}
final_data['label'] = final_data['label'].map(label2class)

In [ ]:
final_data.head()

In [ ]:
# DataFrame 저장
final_data.to_csv('A_comment.csv',index=False)

## 7. X, Y 분리

In [ ]:
# X, Y 분리
features = final_data['data'].values
labels = final_data['label'].values

In [ ]:
features.shape, labels.shape

In [ ]:
# features 내용 3개 출력
features[:3]

In [ ]:
print('이벤트 문자열 최대 길이 :{}'.format(max(len(l) for l in features)))
print('이벤트 문자열 평균 길이 :{}'.format(sum(map(len, features))/len(features)))

In [ ]:
plt.hist([len(s) for s in features], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

## 8. train set와 test set 분리

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels , test_size=0.2, stratify=labels, random_state=41)
x_train.shape, x_test.shape

In [ ]:
# 샘플확인 , 0:긍정, 1:부정
x_train[:2], y_train[:2]

# 반대로 숫자로 단어 매핑
print(tokenizer.index_word)

In [ ]:
# 단어별 빈도수 확인
print(tokenizer.word_counts)

In [ ]:
# 총 단어 갯수 : 1418
max_words = len(tokenizer.index_word)
print(max_words)

## 10. texts_to_sequences : 문장을 숫자로 나열
- 빈도수 적은 단어 제외하는것 없이 모든 단어 포함해서 진행

In [ ]:
# 문장을 숫자로 나열
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

In [ ]:
print(len(x_train_seq), len(x_test_seq))

In [ ]:
print(x_train[1:3])
print(x_train_seq[1:3])

## 11. Padding Sequence

In [ ]:
# 문장의 최대 길이 파악
max(len(line) for line in x_train_seq)

In [ ]:
# 모든 문장을 최대 문장 길이 36에 맞춘다.
x_train_pad = pad_sequences(x_train_seq, maxlen=36)
x_test_pad = pad_sequences(x_test_seq, maxlen=36)

In [ ]:
x_train_pad[:1]

In [ ]:
x_train_pad.shape, x_test_pad.shape

## 12. 모델링

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool2D
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, SimpleRNN, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 하이퍼 파라미터

max_words = 1418 + 1    # 총 단어 갯수 + padding 0 번호
max_len = 36             # 최대 문장 길이
embedding_dim = 32      # embedding 차원

In [ ]:
# 모델 선언
model = Sequential()

In [ ]:
# 단어를 의미있는 32 차원으로 Vector 변경
model.add(Embedding(max_words, embedding_dim, input_length=max_len))

In [ ]:
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(128, activation='swish'))
model.add(Dense(32, activation='swish'))
model.add(Dense(2, activation='softmax'))

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = 'accuracy')
model.summary()

In [ ]:
# 조기종료 콜백함수 정의
es = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 체크포인트 저장
checkpoint_path = 'tmp_checkpoint.ckpt'
cp = ModelCheckpoint(checkpoint_path, save_weights_only=True, monitor='val_loss',
                               verbose=1, save_best_only=True)

In [ ]:
%%time
history = model.fit(x_train_pad, y_train, epochs=50, batch_size=512,
                      validation_split=0.2, verbose =1, callbacks=[es, cp])

In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['accuracy'])
plt.plot(epochs, history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], )
plt.show()

In [ ]:
model.evaluate(x_test_pad, y_test)

In [ ]:
print(f'문자열 : {x_test[0]}')
print(f'Sequence : {x_test_pad[0]}')

In [ ]:
predict = model.predict(x_test_pad[:1])

In [ ]:
print(f'True : {class2label[y_test[0]]}')
print(f'Predict : {class2label[np.argmax(predict)]}')